In [2]:
using ConstLab
using Plots
using Voigt
plotly();

In [ ]:
include("mises_mixed_hardening.jl")

### Parameters

In [8]:
σy =  200.0*10^6
mp = MisesMixedHardMP(
    E = 200.0 * 10^9,
    ν =  0.3,
    σy = σy ,
    H = 0.1 * 200*10^9,
    κ∞ = 0.1 * σy,
    α∞ = 0.1 * σy,
    r = 0.5
);

### Material Status

In [12]:
ms = MisesMixedHardMS()

MisesMixedHardMS
  ₙεₚ: [0.0,0.0,0.0,0.0,0.0,0.0]
  ₙσdev: [0.0,0.0,0.0,0.0,0.0,0.0]
  ₙαdev: [0.0,0.0,0.0,0.0,0.0,0.0]
  ₙκ: 0.0
  ₙμ: 0.0
  loading: false

### Load case: Plain Strain

In [26]:
tend = 1.0
nts = 100
ts = collect(linspace(0, tend, nts))
ε_max = 0.01
εs_pstrain, σs_pstrain, ε_control_pstrain = loadcase(:uniaxial_strain, ε_max, ts);

In [18]:
εs_res_pstrain, σs_res_pstrain, matstats = driver(stress, ATS, ms, mp, ts, εs_pstrain, 
σs_pstrain, ε_control_pstrain);

### Load case: Plain Uniaxial stress

In [27]:
ts = collect(linspace(0, 1, nts))
ε_max = 0.01
εs_pstress, σs_pstress, ε_control_pstress = loadcase(:uniaxial_stress, ε_max, ts);
εs_res_pstress, σs_res_pstress, matstats = driver(stress, ATS,ms, mp, ts, εs_pstress, 
                                                  σs_pstress, ε_control_pstress, ftol = 1e-5);

### Plotting

In [25]:
vm(x) = sqrt(3/2) * vnorm(dev(x));

In [36]:
#p = plot(εs_res_pstrain, σs_res_pstrain, matstats; x_mod = 1, y_mod = 1);
p = plot()
p = plot!(p, vec(εs_res_pstress[1,:]), 
          Float64[vm(σs_res_pstrain[:,i]) for i = 1:nts],
          label="Plane strain");
p = plot!(p, vec(εs_res_pstress[1,:]), 
             Float64[vm(σs_res_pstress[:,i]) for i = 1:nts],
             label="Plane stress");
p = plot!(p, vec(εs_res_pstress[1,:]), 
             (mp.κ∞ + mp.α∞ + mp.σy) * ones(ts),
             label="Saturation stress")
xlabel!(p, "ε_11"); ylabel!(p, "σ"); title!(p, "Uniaxial strain vs stress")